# <font color="blue">LINE Bot 開發</font> <font size=4>by Enos Chou</font>
<br>
<tt>
LINE Bot Official Framework: <a href="https://github.com/line/line-bot-sdk-python"><tt>https://github.com/line/line-bot-sdk-python</tt></a>
<br>
LINE Bot ft. Flask Sample: &nbsp;&nbsp;<a href="https://github.com/line/line-bot-sdk-python/blob/master/examples/flask-kitchensink/app.py"><tt>https://github.com/line/line-bot-sdk-python/blob/master/examples/flask-kitchensink/app.py</tt></a>
</tt>

## Required Packages

In [ ]:
!pip install flask line-bot-sdk

## LINE Bot Official Sample

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token='YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run()

## 啟動 LINE Bot Server

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token='YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=str(event))]
            )
        )

if __name__ == "__main__":
    app.run()

## ImageMessageContent

### 1. 取得圖片儲存至 local

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )
        

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
        
        # packets to local file
        with open(f'{event.message.id}.jpg', 'wb') as f:
            f.write(message_content)
        
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=f'got {event.message.id}.jpg')]
            )
        )

if __name__ == "__main__":
    app.run()

### 2. 取得圖片不儲存

In [ ]:
!pip install pillow

In [ ]:
import json
from io import BytesIO

from flask import Flask, request, abort
from PIL import Image

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_blob_api = MessagingApiBlob(api_client)


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )
        

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_message(event):
    message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
    
    # packets to image directly
    image = Image.open(BytesIO(message_content))
    image.show()
    
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'{event.message.id}')]
        )
    )

if __name__ == "__main__":
    app.run()

### 3. 取得圖片儲存於雲端

In [ ]:
!pip install google-cloud-storage

In [ ]:
import json

from flask import Flask, request, abort
from google.cloud import storage

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_blob_api = MessagingApiBlob(api_client)
service = env['YOUR_SERVICE']
bucket = env['YOUR_BUCKET']


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )
        

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_message(event):
    message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
    
    # packets to GCS
    storage_client = storage.Client.from_service_account_json(service)
    bk = storage_client.bucket(bucket)
    name = event.message.id+'.jpg'
    bk.blob(name).upload_from_string(message_content, content_type='image/jpeg')
    
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text='got '+name)]
        )
    )

if __name__ == "__main__":
    app.run()

## AudioMessageContent

### 取得聲音儲存至 local

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    AudioMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_blob_api = MessagingApiBlob(api_client)


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )
        

@handler.add(MessageEvent, message=AudioMessageContent)
def handle_message(event):
    message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
    
    # packets to local file
    with open(f'{event.message.id}.m4a', 'wb') as f:
        f.write(message_content)
    
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'{event.message.id}')]
        )
    )

if __name__ == "__main__":
    app.run()

## LocationMessageContent

### 取得位置

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
        

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )
        

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_message(event):
    print(event)
    
    r = f'地址: {event.message.address}\n'
    r += f'經度: {event.message.longitude}\n'
    r += f'緯度: {event.message.latitude}'
    
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=r)]
        )
    )

if __name__ == "__main__":
    app.run()

## Reply Message

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    ImageMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

### TextMessage

In [ ]:
TextMessage()

### ImageMessage

In [ ]:
ImageMessage()

#### 回傳圖片自 local

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    ImageMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    base = 'your_base'
    filename = 'longicorn.jpg'
    url = base + app.static_url_path + '/' + filename  # default of app.static_url_path is static
    print(url)
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[ImageMessage(original_content_url=url, preview_image_url=url)]
            )
        )
        

if __name__ == "__main__":
    app.run()

### LocationMessage

In [ ]:
LocationMessage()

#### 回傳位置地圖

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(title='板橋氣象站', address='板橋區', latitude=24.99764722, longitude=121.4420167)]
            )
        )

if __name__ == "__main__":
    app.run()

### FlexMessage

#### Simple Bubble

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with open('bubble.json') as f:
        j = f.read()

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[FlexMessage(altText='Simple Bubble', contents=FlexContainer.from_json(j))]
            )
        )

if __name__ == "__main__":
    app.run()

#### Simple Carousel

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with open('carousel.json', encoding='utf-8') as f:
        j = f.read()

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[FlexMessage(altText='Simple Carousel', contents=FlexContainer.from_json(j))]
            )
        )

if __name__ == "__main__":
    app.run()

## Actions within FlexMessage

### postback

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    PostbackEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with open('postback.json', encoding='utf-8') as f:
        j = f.read()

    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[FlexMessage(altText='action postback', contents=FlexContainer.from_json(j))]
        )
    )

@handler.add(PostbackEvent)
def handle_message(event):
    print(event.postback.data)
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text='got postback')]
        )
    )

        
if __name__ == "__main__":
    app.run()

### uri - camera

In [ ]:
import json
from io import BytesIO

from flask import Flask, request, abort
from PIL import Image

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_blob_api = MessagingApiBlob(api_client)


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with open('camera.json', encoding='utf-8') as f:
        j = f.read()

    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[FlexMessage(altText='action uri', contents=FlexContainer.from_json(j))]
        )
    )

        
@handler.add(MessageEvent, message=ImageMessageContent)
def handle_message(event):
    message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
    
    # packets to image directly
    image = Image.open(BytesIO(message_content))
    image.show()
    
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'{event.message.id}')]
        )
    )
        
if __name__ == "__main__":
    app.run()

### uri - camera roll

In [ ]:
import json
from io import BytesIO

from flask import Flask, request, abort
from PIL import Image

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_blob_api = MessagingApiBlob(api_client)


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with open('cameraroll.json', encoding='utf-8') as f:
        j = f.read()

    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[FlexMessage(altText='action uri', contents=FlexContainer.from_json(j))]
        )
    )

        
@handler.add(MessageEvent, message=ImageMessageContent)
def handle_message(event):
    message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
    
    # packets to image directly
    image = Image.open(BytesIO(message_content))
    image.show()
    
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'{event.message.id}')]
        )
    )
        
if __name__ == "__main__":
    app.run()

### uri - location

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with open('location.json', encoding='utf-8') as f:
        j = f.read()

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[FlexMessage(altText='action uri', contents=FlexContainer.from_json(j))]
            )
        )
        
if __name__ == "__main__":
    app.run()

### uri - web

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with open('web.json', encoding='utf-8') as f:
        j = f.read()

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[FlexMessage(altText='action uri', contents=FlexContainer.from_json(j))]
            )
        )
        
if __name__ == "__main__":
    app.run()

### message

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    if event.message.text == 'test':
        with open('message.json', encoding='utf-8') as f:
            j = f.read()
        msg = FlexMessage(altText='action message', contents=FlexContainer.from_json(j))
    else:
        msg = TextMessage(text=event.message.text)

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[msg]
            )
        )
        
if __name__ == "__main__":
    app.run()

### datetimepicker

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    PostbackEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with open('datetimepicker.json', encoding='utf-8') as f:
        j = f.read()

    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[FlexMessage(altText='action datetimepicker', contents=FlexContainer.from_json(j))]
        )
    )

@handler.add(PostbackEvent)
def handle_message(event):
    print(event.postback.data)
    print(event.postback.params)
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'got {event.postback.params}')]
        )
    )

        
if __name__ == "__main__":
    app.run()

## Quick Reply

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    QuickReply,
    QuickReplyItem,
    MessageAction,
    PostbackAction,
    CameraAction,
    URIAction
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    quick_reply = None
    if text == 'test':
        items = [QuickReplyItem(action=MessageAction(label="message", text="text"), imageUrl="https://i.imgur.com/BB4Js7o.png"),
                 QuickReplyItem(action=PostbackAction(label="postback", data="data")),
                 QuickReplyItem(action=CameraAction(label="camera")),
                 QuickReplyItem(action=URIAction(label="uri", uri="https://developers.line.biz/"))
                ]
        quick_reply = QuickReply(items=items)
        text = 'choose an action'
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=text, quick_reply=quick_reply)]
            )
        )
        
if __name__ == "__main__":
    app.run()

## Rich Menu

### Upload Rich Menu JSON

In [ ]:
import json
from pprint import pprint

import requests

# config
richmenu_json = 'testmenu2.json'
with open('env.json') as f:
    env = json.load(f)

# SPEC of URL
url = 'https://api.line.me/v2/bot/richmenu'
headers = {'Content-Type': 'application/json',
           'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'
          }

# send
with open(richmenu_json, encoding='utf-8') as f:
    rich = f.read()
response = requests.post(url, headers=headers, data=rich.encode('utf-8'))
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response)
response.close()

### Upload Rich Menu Image

In [ ]:
import json
from pprint import pprint

import requests

# config
richmenu_image = 'testmenu2.jpg'
richmenu_id = response.json()['richMenuId']
print(richmenu_id)
with open('env.json') as f:
    env = json.load(f)

# SPEC of URL
url = f'https://api-data.line.me/v2/bot/richmenu/{richmenu_id}/content'
headers = {'Content-Type': 'image/jpeg',
           'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'
          }

# send
with open(richmenu_image, 'rb') as img:
    response = requests.post(url, headers=headers, data=img)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### List Rich Menus

In [ ]:
import json
from pprint import pprint

import requests

# config
with open('env.json') as f:
    env = json.load(f)

# SPEC of URL
url = 'https://api.line.me/v2/bot/richmenu/list'
headers = {'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'}

# send
response = requests.get(url, headers=headers)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### Set Default Rich Menu

In [ ]:
import json
from pprint import pprint

import requests

# config
richmenu_id = 'your_richmenu'
print(richmenu_id)
with open('env.json') as f:
    env = json.load(f)

# SPEC of URL
url = f'https://api.line.me/v2/bot/user/all/richmenu/{richmenu_id}'
headers = {'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'}

# send
response = requests.post(url, headers=headers)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### Get Default Rich Menu

In [ ]:
import json
from pprint import pprint

import requests

# config
with open('env.json') as f:
    env = json.load(f)

# SPEC of URL
url = 'https://api.line.me/v2/bot/user/all/richmenu'
headers = {'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'}

# send
response = requests.get(url, headers=headers)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### Cancel Default Rich Menu

In [ ]:
import json
from pprint import pprint

import requests

# config
with open('env.json') as f:
    env = json.load(f)

# SPEC of URL
url = 'https://api.line.me/v2/bot/user/all/richmenu'
headers = {'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'}

# send
response = requests.delete(url, headers=headers)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### Delete One Rich Menu

In [ ]:
import json
from pprint import pprint

import requests

# config
richmenu_id = 'your_richmenu'
print(richmenu_id)
with open('env.json') as f:
    env = json.load(f)

    
# SPEC of URL
url = f'https://api.line.me/v2/bot/richmenu/{richmenu_id}'
headers = {'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'}

# send
response = requests.delete(url, headers=headers)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### Link Rich Menu to One User

In [ ]:
import json
from pprint import pprint

import requests

# config
richmenu_id = 'your_richmenu'
user_id = 'your_userid'
print(richmenu_id)
print(user_id)
with open('env.json') as f:
    env = json.load(f)


# SPEC of URL
url = f'https://api.line.me/v2/bot/user/{user_id}/richmenu/{richmenu_id}'
headers = {'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'}

# send
response = requests.post(url, headers=headers)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### Get Rich Menu of One User

In [ ]:
import json
from pprint import pprint

import requests

# config
user_id = 'your_userid'
print(user_id)
with open('env.json') as f:
    env = json.load(f)

# SPEC of URL
url = f'https://api.line.me/v2/bot/user/{user_id}/richmenu'
headers = {'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'}

# send
response = requests.get(url, headers=headers)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### Unlink Rich Menu from One User

In [ ]:
import json
from pprint import pprint

import requests

# config
user_id = 'your_userid'
print(user_id)
with open('env.json') as f:
    env = json.load(f)

# SPEC of URL
url = f'https://api.line.me/v2/bot/user/{user_id}/richmenu'
headers = {'Authorization': f'Bearer {env["YOUR_CHANNEL_ACCESS_TOKEN"]}'}

# send
response = requests.delete(url, headers=headers)
print(response.url)
pprint(headers)
if response.status_code == 200:
    pprint(response.json())
else:
    print(response.status_code)
    pprint(response.json())
response.close()

### Personal Rich Menu

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    PostbackEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    richmenu_id = 'your_richmenu'
    if event.message.text == 'Hello':
        line_bot_api.link_rich_menu_id_to_user(user_id=event.source.user_id, rich_menu_id=richmenu_id)
        return
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )

        
@handler.add(PostbackEvent)
def handle_message(event):
    if event.postback.data == 'return':
        line_bot_api.unlink_rich_menu_id_from_user(event.source.user_id)
        return
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text='got postback')]
        )
    )
        
if __name__ == "__main__":
    app.run()

## User Data Handling

### MessagingApi.get_profile(user_id)

In [ ]:
import json
from pprint import pprint

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        pprint(line_bot_api.get_profile(event.source.user_id))
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run()

### FollowEvent and UnfollowEvent

In [ ]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FollowEvent,
    UnfollowEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )

        
@handler.add(FollowEvent)
def handle_follow(event):
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'Welcome {line_bot_api.get_profile(event.source.user_id).display_name}')]
        )
    )


@handler.add(UnfollowEvent)
def handle_unfollow(event):
    print("Got Unfollow event: " + event.source.user_id)

if __name__ == "__main__":
    app.run()

### in JSON

In [ ]:
import json
from time import strftime

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FollowEvent,
    UnfollowEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

try:
    with open('users.json', 'r', encoding='utf-8') as f:
        users = json.loads(f.read())
except:
    users = {}

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )


@handler.add(FollowEvent)
def handle_follow(event):
    profile = line_bot_api.get_profile(event.source.user_id)
    follow(event.source.user_id, profile)
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'Welcome {profile.display_name}')]
        )
    )


@handler.add(UnfollowEvent)
def handle_unfollow(event):
    unfollow(event.source.user_id)


def follow(user_id, profile):
    if user_id in users:
        users[user_id]['follow'] = strftime('%Y/%m/%d-%H:%M:%S')
    else:
        users[user_id] = {'follow': strftime('%Y/%m/%d-%H:%M:%S'),
                          'unfollow': None}
    users[user_id].update(dict(profile))
    output_users()

    
def unfollow(user_id):
    if user_id in users:
        users[user_id]['unfollow'] = strftime('%Y/%m/%d-%H:%M:%S')
        output_users()
    

def output_users():
    with open('users.json', 'w', encoding='utf-8') as f:
        json.dump(users, f)
        
if __name__ == "__main__":
    app.run()

### MongoDB Atlas

In [ ]:
!pip install pymongo

In [ ]:
# verify install OK
from pymongo.mongo_client import MongoClient
uri = "you_mongodb"
client = MongoClient(uri)
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
# create database & collection
database = client['hellodb']
collection = database['hellocollection']

In [ ]:
# add
x = {'_id': 1, 'name': 'Marry', 'height': 160}
y = {'_id': 2, 'name': 'John', 'height': 168, 'weight': 65}
r = collection.insert_one(y)
print(r)

In [ ]:
# update
x = {'height': 170, 'weight': 55}
r = collection.update_one({'_id': 1}, {'$set': x})
print(r)

In [ ]:
# find
import pymongo
list(collection.find().sort('height', pymongo.DESCENDING).skip(1).limit(2))

In [ ]:
# increase count
x = {'weight': 3}
r = collection.update_one({'_id': 1}, {'$inc': x})
print(r)

### in MongoDB Atlas

In [ ]:
!pip install pymongo

In [ ]:
import json
from time import strftime

from flask import Flask, request, abort
from pymongo.mongo_client import MongoClient

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FollowEvent,
    UnfollowEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

try:
    uri = 'your_mongodb'
    client = MongoClient(uri)
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
    db = client['linebot']
    users = db['users']  # collection
except Exception as e:
    print(e)

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )


@handler.add(FollowEvent)
def handle_follow(event):
    profile = line_bot_api.get_profile(event.source.user_id)
    follow(event.source.user_id, profile)
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'Welcome {profile.display_name}')]
        )
    )


@handler.add(UnfollowEvent)
def handle_unfollow(event):
    unfollow(event.source.user_id)


def follow(user_id, profile):
    if users.find_one({'_id': user_id}):
        user = {'follow': strftime('%Y/%m/%d-%H:%M:%S')}
        user.update(dict(profile))
        users.update_one({'_id': user_id}, {'$set': user})
    else:
        user = {'_id': user_id,
                'follow': strftime('%Y/%m/%d-%H:%M:%S'),
                'unfollow': None}
        user.update(dict(profile))
        users.insert_one(user)             

    
def unfollow(user_id):
    if users.find_one({'_id': user_id}):
        users.update_one({'_id': user_id}, {'$set': {'unfollow': strftime('%Y/%m/%d-%H:%M:%S')}})
        
if __name__ == "__main__":
    app.run()

### Google Cloud Firestore

In [ ]:
!pip install google-cloud-firestore

In [ ]:
# verify install OK
from google.cloud import firestore
try:
    client = firestore.Client.from_service_account_json(env['YOUR_SERVICE'])
except Exception as e:
    print(e)

In [ ]:
# create collection
hello = client.collection('hello')

In [ ]:
# add
x = {'id': '1', 'name': 'Marry', 'height': 160}
y = {'id': '2', 'name': 'John', 'height': 168, 'weight': 65}
r = hello.add(x, document_id='1')
r = hello.add(y, document_id='2')
print(r)

In [ ]:
# update
x = {'height': 170, 'weight': 55}
r = hello.document('1').update(x)
print(r)

In [ ]:
# find
for r in hello.order_by('height', direction='DESCENDING').offset(1).limit(2).get():
    print(r.to_dict())

In [ ]:
# increase count
hello.document('2').update({'weight': firestore.Increment(3)})

### in Google Cloud Firestore

In [ ]:
!pip install google-cloud-firestore

In [ ]:
import json
from time import strftime

from flask import Flask, request, abort
from google.cloud import firestore

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FollowEvent,
    UnfollowEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

try:
    client = firestore.Client.from_service_account_json(env['YOUR_SERVICE'])
    users = client.collection('users')
except Exception as e:
    print(e)

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )


@handler.add(FollowEvent)
def handle_follow(event):
    profile = line_bot_api.get_profile(event.source.user_id)
    follow(event.source.user_id, profile)
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'Welcome {profile.display_name}')]
        )
    )


@handler.add(UnfollowEvent)
def handle_unfollow(event):
    unfollow(event.source.user_id)


def follow(user_id, profile):
    user = dict(profile)
    if users.document(user_id).get().exists:
        user.update({'follow': strftime('%Y/%m/%d-%H:%M:%S')})
        users.document(user_id).update(user)
    else:
        user.update({'follow': strftime('%Y/%m/%d-%H:%M:%S'), 'unfollow': None})
        users.add(document_data=user, document_id=user_id)               

    
def unfollow(user_id):
    user = users.document(user_id)
    if user.get().exists:
        user.update({'unfollow': strftime('%Y/%m/%d-%H:%M:%S')})
        
if __name__ == "__main__":
    app.run()

## Push Message

### push_message

In [ ]:
import json

from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    TextMessage,
    PushMessageRequest
)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])

userid = 'your_userid'

with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_api.push_message(
        PushMessageRequest(
            to=userid,
            messages=[TextMessage(text='PUSH!')]
        )
    )

### multicast

In [ ]:
import json

from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    TextMessage,
    MulticastRequest
)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])

userids = ['your_userid',
           'your_userid']

with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_api.multicast(
        MulticastRequest(
            to=userids,
            messages=[TextMessage(text='Multicast!')]
        )
    )

### broadcast

In [ ]:
import json

from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    TextMessage,
    BroadcastRequest
)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])

with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_api.broadcast(
        BroadcastRequest(
            messages=[TextMessage(text='Broadcast!')]
        )
    )